In [1]:
from rdflib import Graph,URIRef,Namespace,BNode,Literal
from rdflib.namespace import XSD,RDF
import pandas as pd
import argparse
from collections import defaultdict
from datetime import datetime
import json
from rdflib import URIRef
from collections import defaultdict
import json



In [2]:
graph = Graph()
directory = 'sampleTSS.ttl'
print("Started loading graph...")
graph.parse(directory, format="turtle",publicID="https://example.org/")
print("Graph loaded successfully.")

Started loading graph...
Graph loaded successfully.


In [ ]:

#Get unique time series snippets
snippet_set = set()

get_snippet_query = '''
        PREFIX sosa: <http://www.w3.org/ns/sosa/>
        PREFIX tss:  <https://w3id.org/tss#>
        PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

        SELECT DISTINCT ?snippet ?PtsObj
        WHERE{
            ?snippet a tss:Snippet .
            ?snippet tss:points ?PtsObj .
        }
        '''
print('Started identifying unique sensors')
for snippet, PtsObj in graph.query(get_snippet_query):
    snippet_set.add(snippet)   # keep adding the snippet URI
    print(f"Snippet: {snippet}, PtsObj: {PtsObj}")
    print(type(PtsObj))

print('Snippets identified successfully')


SyntaxError: invalid syntax (2296010491.py, line 24)

In [ ]:
# Nested dict: subject -> predicate -> list of objects
results = defaultdict(lambda: defaultdict(list))

#Create RDF
prefix_tss = Namespace('https://w3id.org/tss#')
prefix_ex  = Namespace('http://example.org/')
prefix_sosa  = Namespace('http://www.w3.org/ns/sosa/')
prefix_xsd  = Namespace('http://www.w3.org/2001/XMLSchema#')
final_graph = Graph()
final_graph.bind('tss', prefix_tss)
final_graph.bind('ex', prefix_ex)
final_graph.bind('sosa', prefix_sosa)
final_graph.bind('xsd', prefix_xsd)
print('Started creating final graph')


get_snippet_query = '''
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX tss:  <https://w3id.org/tss#>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

SELECT ?snippet ?P ?O ?about ?aboutP ?aboutO
WHERE {
  ?snippet a tss:Snippet .
  ?snippet ?P ?O .

  OPTIONAL {
    ?snippet tss:about ?about .
    ?about ?aboutP ?aboutO .
  }
}

        '''

for subj, pred, obj, about, aboutP, aboutO in graph.query(get_snippet_query):
    # store snippet triples
    if subj and pred and obj:
        results[subj][pred].append(obj)
        final_graph.add((subj, pred, obj))

    # store about triples
    if about and aboutP and aboutO:
        results[subj][aboutP].append(aboutO)
        final_graph.add((about, aboutP, aboutO))


Started creating final graph


In [11]:
for subj, pred, obj in final_graph:
    print(subj, pred, obj)

n8c243b20f7a34baab12d33792bc2d50cb2 http://www.w3.org/ns/sosa/observedProperty https://example.org/property/salinity
https://example.org/snippet/2026-01-09-noise-1 https://w3id.org/tss#context {
    "@context": {
      "id": "@id",
      "time": { "@id": "http://www.w3.org/ns/sosa/resultTime", "@type": "http://www.w3.org/2001/XMLSchema#dateTime" },
      "value": { "@id": "http://www.w3.org/ns/sosa/hasSimpleResult", "@type": "http://www.w3.org/2001/XMLSchema#double" }
    }
  }
n8c243b20f7a34baab12d33792bc2d50cb13 http://www.w3.org/ns/sosa/madeBySensor https://example.org/sensor/cod_sensor_1
https://example.org/snippet/2026-01-14-orp-1 https://w3id.org/tss#until 2026-01-15T00:00:00+00:00
https://example.org/snippet/2026-01-11-flow-1 https://w3id.org/tss#about n8c243b20f7a34baab12d33792bc2d50cb11
https://example.org/snippet/2026-01-12-level-1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/tss#Snippet
https://example.org/snippet/2026-01-01-temp-1 https://w3id.org/tss#fr